# Map Reduce Paradigm

Lacking a database, a common way to build a dataset is with the Map-Filter-Reduce paradigm.

- Map transforms the database
- Filter is a where clause to subselect the data
- Reduce is effectively a GroupBy operation, folding data into aggregates.

Let's look on an applied task:

Given a list of strings, return the longest string

In [2]:
def find_longest_string(list_of_strings):
    longest_string = None
    longest_string_len = 0     
    for s in list_of_strings:
        if len(s) > longest_string_len:
            longest_string_len = len(s)
            longest_string = s
    return longest_string

list_of_strings = ['abc', 'python', 'dima'] * 1000
%time max_length = print(find_longest_string(list_of_strings))

python
CPU times: user 457 µs, sys: 198 µs, total: 655 µs
Wall time: 475 µs


break our code into two steps: 1) compute the len of all strings and 2) select the max value

In [3]:
# step 1:
list_of_string_lens = [len(s) for s in list_of_strings]
list_of_string_lens = zip(list_of_strings, list_of_string_lens)
#step 2:
max_len = max(list_of_string_lens, key=lambda t: t[1])
print(max_len)

('python', 6)


Call “step one” a “mapper” because it maps some value into some other value

call “step two” a reducer because it gets a list of values and produces a single (in most cases) value. 

In [4]:
mapper = len

def reducer(p, c):
    """
    Takes two tuples as input 
    returns the one with the biggest len
    """
    if p[1] > c[1]:
        return p
    return c

Step 1 maps our list of strings into a list of tuples using the mapper function (here I use the zip again to avoid duplicating the strings).

Step 2 uses the reducer function, goes over the tuples from step one and applies it one by one. The result is a tuple with the maximum length.

In [5]:
from functools import reduce
#step 1
mapped = map(mapper, list_of_strings)
mapped = zip(list_of_strings, mapped)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)


In [6]:
import numpy as np

data_chunks = np.split(np.array(list_of_strings), 30)
#step 1:
reduced_all = []
for chunk in data_chunks:
    mapped_chunk = map(mapper, chunk)
    mapped_chunk = zip(chunk, mapped_chunk)
    reduced_chunk = reduce(reducer, mapped_chunk)
    reduced_all.append(reduced_chunk)
    
#step 2:
reduced = reduce(reducer, reduced_all)

In step one, we go over our chunks and find the longest string in that chunk using a map and reduce. 

In step two, we take the output of step one, which is a list of reduced values, and perform a final reduce to get the longest string.

In [21]:
def chunks_mapper(chunk):
    mapped_chunk = map(mapper, chunk) 
    mapped_chunk = zip(chunk, mapped_chunk)
    return reduce(reducer, mapped_chunk)
    
%time

data_chunks = np.split(np.array(list_of_strings), 30)
#step 1:
mapped = map(chunks_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 12.2 µs
('python', 6)


In [23]:
from multiprocessing import Pool

N_THREADS = 7

pool = Pool(N_THREADS)
data_chunks = np.array_split(np.array(list_of_strings), N_THREADS)
#step 1:
mapped = pool.map(chunks_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)


In [26]:
from joblib import Parallel, delayed

N_THREADS = 7

pool = Pool(N_THREADS)
data_chunks = np.array_split(np.array(list_of_strings), N_THREADS)
#step 1:
mapped = Parallel(n_jobs=N_THREADS)(delayed(chunks_mapper)(x) for x in data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)


Our architecture is built using two functions: map and reduce . Each computation unit maps the input data and executes the initial reduce. Finally, some centralized unit executes the final reduce and returns the output. It looks like this:

![](mapreduce.png)